In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import ensembleReader as er
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
# import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
from pathlib import Path
# reload(psce)
from keras.utils import multi_gpu_model
import keras
import keras.optimizers
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss

Using TensorFlow backend.


In [2]:
train_data = pkl.load(open("/n/scratch2/ms994/train_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [3]:
test_data =  pkl.load(open("/n/scratch2/ms994/test_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [4]:
valid_data = pkl.load(open("/n/scratch2/ms994/valid_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [5]:
sum([train_datum[1][0] for train_datum in train_data])/len(train_data)

0.08077938064363824

In [6]:
def patient_func(tkn_file_paths, session_instead_patient):
        return [read.parse_edf_token_path_structure(tkn_file_path)[1] + "/" + read.parse_edf_token_path_structure(tkn_file_path)[2] for tkn_file_path in tkn_file_paths]

In [7]:
seizureInfo = pd.read_csv("/home/ms994/dbmi_eeg_clustering/assets/seizures.csv")

In [8]:
seizureInfo.groupby("seizure_type")["seizure_type"].count() 

seizure_type
ABSZ      99
CPSZ     361
FNSZ    1834
GNSZ     585
MYSZ       3
SPSZ      52
TCSZ      53
TNSZ      68
Name: seizure_type, dtype: int64

Use the top 3 seizures only, CPSZ, FNSZ, and GNSZ. only include a patient if they are characterized by the seizure or have no seizure in the set. It is either unfair/incorrect/too hard to include the healthy state of patients not in this subpopulation

In [9]:
noSeizureFiles = seizureInfo[pd.isnull(seizureInfo.seizure_type)].fn

In [10]:
seizurePatientMap = seizureInfo.groupby(["patient", "seizure_type"])["patient", "seizure_type"].count()

In [11]:
seizurePatientMap.columns = ["patient_cnt", "seizure_type_cnt"]

In [12]:
seizurePatientMap = seizurePatientMap.reset_index()

In [13]:
matchingPatients = seizurePatientMap[seizurePatientMap.seizure_type.isin(["CPSZ", "FNSZ", "GNSZ"])].patient

In [14]:
noSeizurepatients = [read.parse_edf_token_path_structure(file)[1] for file in noSeizureFiles]

In [15]:
matchingPatients = matchingPatients.apply(lambda a: "{:08d}".format(int(a))).tolist()

In [16]:
toFilterBy = noSeizurepatients + matchingPatients

In [17]:
"{}/{}".format(read.parse_edf_token_path_structure(noSeizureFiles[0])[1], read.parse_edf_token_path_structure(noSeizureFiles[0])[2])

'00000258/s002_2003_07_21'

In [18]:
(train_data[0][0].std(axis=0) > 100).any()

False

In [19]:
def update_data(edss,  seizure_classes_to_use=["bckg", "gnsz", "fnsz", "cpsz"], include_seizure_type=True, include_montage_channels=True, remove_outlier_by_std_thresh=100, zero_center_over_time=True, is_eval_set=False):
    '''
    since we store the full string of the session or the patient instead of the index, we update the data to use the int index
    some of the tasks require different datasets and some filtering of the data i.e. only seizure classification or just some of the labels
    is_eval_set: for use with valid and test set, since they shouldn't have patients from the train set and therefore predicting for them is wrong
    '''

    data = [datum[0] for datum in edss]
    if is_eval_set:
        patient_labels = [0 for i in range(len(edss))]
        remove_outlier_by_std_thresh=None
    else:
        patients = [datum[1][1] for datum in edss]
        allPatient = list(set(patients))
        patient_labels = [allPatient.index(patient) for patient in patients]
    seizure_detection_labels = [datum[1][0] for datum in edss]
    if include_seizure_type:
        seizure_class_labels = [datum[1][2] for datum in edss]
    if include_montage_channels:
        montage_channel_labels = [datum[1][3] for datum in edss]
    keep_index = [True for i in range(len(data))]
    if seizure_classes_to_use is not None:
        for i, seizure_class_label in enumerate(seizure_class_labels):
            if seizure_detection_labels[i] and constants.SEIZURE_SUBTYPES[seizure_class_label] not in seizure_classes_to_use: #it's a seizure detection class and it should be excluded
                keep_index[i] = False
    for i, patient_session in enumerate(patients):
        if patient_session.split("/")[0] not in toFilterBy:
            keep_index[i] = False
        

    if zero_center_over_time:
        for i, datum in enumerate(data):
            data[i] = (datum.T - datum.mean(1)).T
    if remove_outlier_by_std_thresh is not None:
        removed = 0
        for i, datum in enumerate(data):
            if (np.std(datum, axis=0) > remove_outlier_by_std_thresh).any():
                keep_index[i] = False
                removed += 1
        print("We removed {} because unclean by std".format(removed))


    data_to_keep = []
    patient_labels_to_keep = []
    seizure_detect_to_keep = []
    seizure_class_labels_to_keep = []
    montage_channel_labels_to_keep = []

    for i, should_keep in enumerate(keep_index):
        if should_keep:
            data_to_keep.append(data[i])
            patient_labels_to_keep.append(patient_labels[i])
            seizure_detect_to_keep.append(seizure_detection_labels[i])
            if include_montage_channels:
                montage_channel_labels_to_keep.append(montage_channel_labels[i])
            if include_seizure_type:
                seizure_class_labels_to_keep.append(seizure_class_labels[i])
    return [(data_to_keep[i], (seizure_detect_to_keep[i], patient_labels_to_keep[i], seizure_class_labels_to_keep[i], montage_channel_labels_to_keep[i].values)) for i in range(len(data_to_keep))], seizure_detect_to_keep, patient_labels_to_keep, seizure_class_labels_to_keep, montage_channel_labels_to_keep

In [20]:
constants.SEIZURE_SUBTYPES

['bckg',
 'fnsz',
 'gnsz',
 'spsz',
 'cpsz',
 'absz',
 'tnsz',
 'cnsz',
 'tcsz',
 'atsz',
 'mysz']

In [21]:
train_data = update_data(train_data)

We removed 32959 because unclean by std


In [22]:
sum(train_data[1])

15389

In [23]:
train_data = train_data[0]

In [24]:
sum([train_datum[1][0] for train_datum in train_data])

15389

In [25]:
valid_data = update_data(valid_data, remove_outlier_by_std_thresh=None)
test_data = update_data(test_data, remove_outlier_by_std_thresh=None)

In [26]:
valid_data = valid_data[0]
test_data = test_data[0]

In [27]:
sum([valid_datum[1][0] for valid_datum in valid_data]), sum([test_datum[1][0] for test_datum in test_data])

(4696, 12273)

In [28]:
from random import shuffle

In [29]:
shuffle(train_data)
shuffle(valid_data)
shuffle(test_data)

In [30]:
import tensorflow as tf
# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_feature_list(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
# Helperfunctions to make your feature definition more readable
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _float_feature_list(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [31]:
valid_data[0][1][3].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
is_train = True
def parse_datum(datum):
    global is_train
    xData = datum[0]
    yData = datum[1]
    
    feature = { \
               'data': _float_feature_list(xData.reshape(-1)), \
               'label': _int64_feature_list([yData[0]]), \
               'subtypeLabel': _int64_feature_list([yData[2]]), \
               'session': _int64_feature(yData[1]), \
               'montage': _int64_feature_list(yData[3].astype(int))
              }
    return tf.train.Example(features=tf.train.Features(feature=feature))


In [34]:
pkl.dump(train_data, open("/n/scratch2/ms994/train_montage_subtype.pkl", "wb"))
pkl.dump(valid_data, open("/n/scratch2/ms994/valid_montage_subtype.pkl", "wb"))
pkl.dump(test_data, open("/n/scratch2/ms994/test_montage_subtype.pkl", "wb"))

In [38]:
def get_gen(dataset):
    def gen():
        for i in range(len(dataset)):
            yield parse_datum(dataset[i]).SerializeToString()
    return gen

In [39]:
dataset = tf.data.Dataset.from_generator(get_gen(train_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))

In [40]:

dataset = tf.data.Dataset.from_generator(get_gen(train_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/train_4s.tfr")
writer.write(dataset)

In [41]:
dataset = tf.data.Dataset.from_generator(get_gen(valid_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/valid_4s.tfr")
writer.write(dataset)

In [42]:
dataset = tf.data.Dataset.from_generator(get_gen(test_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/test_4s.tfr")
writer.write(dataset)

# try to write an image file down

In [29]:
del valid_data
del test_data

In [30]:
import math as m
import numpy as np
import scipy.io
from scipy.interpolate import griddata
from sklearn.preprocessing import scale
from functools import reduce
def gen_images(locs, features, n_gridpoints=32, normalize=True, edgeless=False):
    """
    Generates EEG images given electrode locations in 2D space and multiple feature values for each electrode
    :param locs: An array with shape [n_electrodes, 2] containing X, Y
                        coordinates for each electrode.
    :param features: Feature matrix as [n_samples, n_features]
                                Features are as columns.
                                Features corresponding to each frequency band are concatenated.
                                (alpha1, alpha2, ..., beta1, beta2,...)
    :param n_gridpoints: Number of pixels in the output images
    :param normalize:   Flag for whether to normalize each band over all samples
    :param edgeless:    If True generates edgeless images by adding artificial channels
                        at four corners of the image with value = 0 (default=False).
    :return:            Tensor of size [samples, colors, W, H] containing generated
                        images.
    """
    feat_array_temp = []
    nElectrodes = locs.shape[0]     # Number of electrodes
    # Test whether the feature vector length is divisible by number of electrodes
    assert features.shape[1] % nElectrodes == 0
    n_colors = features.shape[1] // nElectrodes
    for c in range(n_colors):
        feat_array_temp.append(features[:, c * nElectrodes : nElectrodes * (c+1)])  # features.shape为[samples, 3*nElectrodes]

    nSamples = features.shape[0]    # sample number 2670
    # Interpolate the values        # print(np.mgrid[-1:1:5j]) get [-1.  -0.5  0.   0.5  1. ]
    grid_x, grid_y = np.mgrid[
                     min(locs[:, 0]):max(locs[:, 0]):n_gridpoints*1j,
                     min(locs[:, 1]):max(locs[:, 1]):n_gridpoints*1j
                     ]
    
    temp_interp = []
    for c in range(n_colors):
        temp_interp.append(np.zeros([nSamples, n_gridpoints, n_gridpoints]))

    
    # Generate edgeless images
    if edgeless:
        min_x, min_y = np.min(locs, axis=0)
        max_x, max_y = np.max(locs, axis=0)
        locs = np.append(locs, np.array([[min_x, min_y], [min_x, max_y],[max_x, min_y],[max_x, max_y]]),axis=0)
        for c in range(n_colors):
            feat_array_temp[c] = np.append(feat_array_temp[c], np.zeros((nSamples, 4)), axis=1)
    
    # Interpolating
    for i in range(nSamples):
        for c in range(n_colors):
            temp_interp[c][i, :, :] = griddata(locs, feat_array_temp[c][i, :], (grid_x, grid_y),    # cubic
                                    method='cubic', fill_value=np.nan)
    
#     # Normalizing
#     for c in range(n_colors):
#         if normalize:
#             temp_interp[c][~np.isnan(temp_interp[c])] = \
#                 scale(temp_interp[c][~np.isnan(temp_interp[c])])
        
#         temp_interp[c] = np.nan_to_num(temp_interp[c])
        
    temp_interp = np.swapaxes(np.asarray(temp_interp), 0, 1)     # swap axes to have [samples, colors, W, H] # WH xy
    temp_interp = np.swapaxes(temp_interp, 1, 2)
    temp_interp = np.swapaxes(temp_interp, 2, 3)    # [samples, W, H，colors]
    return temp_interp
def cart2sph(x, y, z):
    """
    Transform Cartesian coordinates to spherical
    :param x: X coordinate
    :param y: Y coordinate
    :param z: Z coordinate
    :return: radius, elevation, azimuth
    """
    x2_y2 = x**2 + y**2
    r = m.sqrt(x2_y2 + z**2)                    # r     tant^(-1)(y/x)
    elev = m.atan2(z, m.sqrt(x2_y2))            # Elevation
    az = m.atan2(y, x)                          # Azimuth
    return r, elev, az


def pol2cart(theta, rho):
    """
    Transform polar coordinates to Cartesian 
    :param theta: angle value
    :param rho: radius value
    :return: X, Y
    """
    return rho * m.cos(theta), rho * m.sin(theta)

def azim_proj(pos):
    """
    Computes the Azimuthal Equidistant Projection of input point in 3D Cartesian Coordinates.
    Imagine a plane being placed against (tangent to) a globe. If
    a light source inside the globe projects the graticule onto
    the plane the result would be a planar, or azimuthal, map
    projection.
    :param pos: position in 3D Cartesian coordinates    [x, y, z]
    :return: projected coordinates using Azimuthal Equidistant Projection
    """
    [r, elev, az] = cart2sph(pos[0], pos[1], pos[2])
    return pol2cart(az, m.pi / 2 - elev)

In [31]:
locations = pd.read_csv("../assets/3d_positions.csv")

In [32]:
projections = pd.DataFrame(locations[["x", "y", "z"]].apply(lambda x: azim_proj(x), axis=1).tolist(), columns=["x_proj", "y_proj"])

In [33]:
projections.loc[:,"channels"] = locations.label

In [34]:
def gen_image_from_snapshot(snapshot):
    return np.nan_to_num(gen_images(projections[["x_proj", "y_proj"]].values, snapshot.reshape(1,21),  n_gridpoints=16).reshape((16,16)))

In [35]:
xDatum = train_data[0][0]

In [37]:
gen_image_from_snapshot(xDatum[0]).shape

(16, 16)

In [47]:
from multiprocessing import Process, Manager
m = Manager()
toRunQ = m.Queue()
toReturnQ = m.Queue()

In [48]:
def transform_xy_to_image(toRunQ, toReturnQ):
    for i in iter(toRunQ.get, None):
        x, y = train_data[i]
        x = np.apply_along_axis(gen_image_from_snapshot, 1, x)
        toReturnQ.put((x,y))

In [49]:
n_process = 12
processes = [Process(target=transform_xy_to_image, args=(toRunQ, toReturnQ)) for i in range(n_process)]
[toRunQ.put(i) for i in range(len(train_data))]
[toRunQ.put(None) for i in range(n_process)]
print("setup")

setup


In [50]:
[p.start() for p in processes]

[None, None, None, None, None, None, None, None, None, None, None, None]

Process Process-43:
Process Process-46:
Process Process-45:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-48-9f022432ee96>", line 5, in transform_xy_to_image
    toReturnQ.

BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-48-9f022432ee96>", line 5, in transform_xy_to_image
    toReturnQ.put((x,y))
  File "<ipython-input-48-9f022432ee96>", line 5, in transform_xy_to_image
    toReturnQ.put((x,y))
  File "<string>", line 2, in put
  File "<string>", line 2, in put
  File "/home/ms994/miniconda3/envs/keras-redo/lib/python3.7

In [65]:
toRunQ.qsize()

203689

In [41]:
is_train = True
def parse_datum(datum):
    global is_train
    xData = datum[0]
    xData = np.apply_along_axis(gen_image_from_snapshot, 1, xData)
    yData = datum[1]
    
    feature = { \
               'data': _float_feature_list(xData.reshape(-1)), \
               'label': _int64_feature_list([yData[0]]), \
               'subtypeLabel': _int64_feature_list([yData[2]]), \
               'session': _int64_feature(yData[1]), \
               'montage': _int64_feature_list(yData[3].astype(int))
              }
    return tf.train.Example(features=tf.train.Features(feature=feature))


In [43]:
def get_gen(dataset):
    def gen():
        for i in range(len(dataset)):
            yield parse_datum(dataset[i]).SerializeToString()
    return gen

In [44]:
dataset = tf.data.Dataset.from_generator(get_gen(train_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))

In [ ]:

dataset = tf.data.Dataset.from_generator(get_gen(train_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/train_image_4s.tfr")
writer.write(dataset)

In [ ]:
dataset = tf.data.Dataset.from_generator(get_gen(valid_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/valid_image_4s.tfr")
writer.write(dataset)

In [ ]:
dataset = tf.data.Dataset.from_generator(get_gen(test_data), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/test_image_4s.tfr")
writer.write(dataset)

In [ ]:
%%bash
scancel 2810482